In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

data = {}
p = Path(r'D:\ML finance\MLP\data')

for x in p.rglob("*"):
    data[str(x).split('.')[0].split('\\')[-1]] = pd.read_csv(x)

X = pd.concat(data, axis=1)

X.columns = ['_'.join(col).strip() if col[1] else col[0] for col in X.columns.values]
openTime_col = [col for col in X.columns if 'openTime' in col][0]
X.index = pd.to_datetime(X[openTime_col])
X = X.drop(columns=[col for col in X.columns if 'openTime' in col or 'Unnamed' in col])

In [28]:
import pandas as pd
import numpy as np
from pathlib import Path

data = {}
p = Path(r'D:\ML finance\MLP\data')
for x in p.rglob("*"):
    data[str(x).split('.')[0].split('\\')[-1]] = pd.read_csv(x)

X = pd.concat(data, axis=1)

In [30]:
def flatten_columns(multiindex_cols):
    flat_cols = []
    for metric, ticker in multiindex_cols:
        metric_clean = metric.replace('_in_sample', '')
        flat_cols.append(f'{ticker}_{metric_clean}')
    return flat_cols

X.columns = flatten_columns(X.columns)

openTime_cols = [col for col in X.columns if 'openTime' in col]

if openTime_cols:
    openTime = pd.to_datetime(X[openTime_cols[0]])
    X.index = openTime
    cols_to_drop = [col for col in X.columns if 'openTime' in col or 'Unnamed' in col]
    X = X.drop(columns=cols_to_drop)
else:
    raise ValueError("No 'openTime' column found. Check your flattening logic.")


In [32]:
all_tickers = sorted(set([col.rsplit('_', 1)[0] for col in X.columns]))

tickers_to_keep = []
tickers_to_remove = []

for ticker in all_tickers:
    ticker_cols = [col for col in X.columns if col.startswith(f'{ticker}_')]
    
    has_nan = X[ticker_cols].isna().any().any()
    
    if has_nan:
        tickers_to_remove.append(ticker)
    else:
        tickers_to_keep.append(ticker)

print(f"✓ Удалено тикеров: {len(tickers_to_remove)}")
print(f"✓ Осталось тикеров: {len(tickers_to_keep)}")

if tickers_to_remove:
    print(f"\n Удалённые тикеры (есть NaN): {tickers_to_remove[:10]}{'...' if len(tickers_to_remove) > 10 else ''}")


nan_columns = X.columns[X.isna().any()].tolist()
if nan_columns:
    print(f"\n Осталось колонок с NaN: {len(nan_columns)}")
    X = X.drop(columns=nan_columns)
else:
    print(f"\n Колонок с NaN не осталось")


print(f"\n Итоговый размер: {X.shape}")
print(f"✓ Строк: {X.shape[0]}")
print(f"✓ Столбцов: {X.shape[1]}")
print(f"✓ Тикеров: {len(tickers_to_keep)}")


✓ Удалено тикеров: 142
✓ Осталось тикеров: 99

 Удалённые тикеры (есть NaN): ['1000SATSUSDT', 'ACEUSDT', 'ACHUSDT', 'AEVOUSDT', 'AGIXUSDT', 'AGLDUSDT', 'AIUSDT', 'ALTUSDT', 'AMBUSDT', 'APEUSDT']...

 Осталось колонок с NaN: 1136

 Итоговый размер: (17369, 792)
✓ Строк: 17369
✓ Столбцов: 792
✓ Тикеров: 99


In [35]:
agg_dict = {}
for col in X.columns:
    if '_open' in col:
        agg_dict[col] = 'first'
    elif '_high' in col:
        agg_dict[col] = 'max'
    elif '_low' in col:
        agg_dict[col] = 'min'
    elif '_close' in col:
        agg_dict[col] = 'last'
    elif 'volume' in col or 'ntrades' in col:
        agg_dict[col] = 'sum'
    else:
        agg_dict[col] = 'mean'

X_4h = X.resample('4h').agg(agg_dict)

In [36]:
X_4h

,1INCHUSDT_bvolume,AAVEUSDT_bvolume,ADAUSDT_bvolume,ALGOUSDT_bvolume,ALICEUSDT_bvolume,ALPHAUSDT_bvolume,ANKRUSDT_bvolume,ATOMUSDT_bvolume,AUDIOUSDT_bvolume,AVAXUSDT_bvolume,...,WAVESUSDT_takerbuyqvolume,XEMUSDT_takerbuyqvolume,XLMUSDT_takerbuyqvolume,XRPUSDT_takerbuyqvolume,XTZUSDT_takerbuyqvolume,YFIUSDT_takerbuyqvolume,ZECUSDT_takerbuyqvolume,ZENUSDT_takerbuyqvolume,ZILUSDT_takerbuyqvolume,ZRXUSDT_takerbuyqvolume
openTime_bvolume,,,,,,,,,,,,,,,,,,,,,
2021-08-24 00:00:00+00:00,826561.0,13065.4,26091616.0,1824720.8,818614.4,2742100.0,19150702.0,207310.52,647332.0,898177.0,...,6.621468e+05,6.946013e+05,1.304154e+06,2.533061e+07,2.034980e+06,1.330260e+06,7.573191e+05,5.900571e+05,1.433477e+06,8.971720e+05
2021-08-24 04:00:00+00:00,6823890.0,54615.9,128449316.0,13460007.6,3388458.9,14173962.0,56582123.0,1199854.82,2494768.0,3453898.0,...,2.467045e+06,2.594901e+06,8.347913e+06,1.067105e+08,3.693269e+07,2.509681e+06,3.747290e+06,2.885864e+06,5.367356e+06,3.350649e+06
2021-08-24 08:00:00+00:00,4847124.0,83707.9,139359300.0,20817721.4,3986929.7,21037049.0,62473851.0,1009370.19,2820947.0,2939937.0,...,3.940882e+06,2.583687e+06,1.171750e+07,1.389225e+08,1.894435e+07,4.717071e+06,4.388457e+06,3.146891e+06,9.173914e+06,2.419199e+06
2021-08-24 12:00:00+00:00,8850963.0,145653.2,463846787.0,27308757.5,6910072.0,32343293.0,122206484.0,2301745.26,8301095.0,10144885.0,...,5.446789e+06,4.911075e+06,1.741575e+07,2.824625e+08,2.464563e+07,6.421353e+06,7.171793e+06,7.087752e+06,9.071029e+06,4.228093e+06
2021-08-24 16:00:00+00:00,8697683.0,123565.8,461188107.0,18794652.6,6078900.3,39660907.0,89971644.0,1648676.68,6089185.0,7246395.0,...,5.857725e+06,4.773810e+06,1.818901e+07,2.712152e+08,1.850510e+07,6.358535e+06,7.232579e+06,4.587253e+06,7.706418e+06,5.045037e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-17 00:00:00+00:00,20925158.0,141770.4,127684457.0,86149464.1,2693428.4,25581057.0,135514971.0,1183885.02,13596843.0,1602015.0,...,5.289361e+06,5.369155e+06,4.726079e+06,6.575686e+07,1.760532e+06,1.656417e+06,1.008039e+06,1.012958e+06,1.790173e+06,1.247215e+06
2023-08-17 04:00:00+00:00,10497968.0,75283.3,47616528.0,28903202.8,1191393.7,12215091.0,51458022.0,550360.45,6496040.0,660196.0,...,2.755119e+06,1.127096e+06,2.208463e+06,3.519824e+07,6.397020e+05,5.703780e+05,3.291407e+05,3.139347e+05,5.016346e+05,4.577006e+05
2023-08-17 08:00:00+00:00,11501157.0,95149.7,66966323.0,45202996.1,1662726.4,18196440.0,61877965.0,4871864.62,9191698.0,831370.0,...,3.330243e+06,2.966667e+06,2.866337e+06,4.499254e+07,7.472182e+05,6.809703e+05,3.317758e+05,3.643980e+05,9.522406e+05,6.075550e+05


In [40]:
close_cols = [col for col in X_4h.columns if col.endswith('_close')]

print(f"✓ Найдено close-колонок: {len(close_cols)}")

for col in close_cols:
    ticker = col.replace('_close', '')
    ret_col = f'{ticker}_ret'
    X_4h[ret_col] = X_4h[col].pct_change(fill_method=None).shift(-1)

✓ Найдено close-колонок: 99


In [42]:
X_4h.to_csv('data_t.csv', index_label='openTime')